In [8]:
!pip install pyldavis
import pandas as pd 
# if running on a linux/mac, uncomment the wget ones and comment out the curls
#wget https://raw.githubusercontent.com/tvdermeer/NLP/main/stop_words.txt
#wget https://raw.githubusercontent.com/tvdermeer/NLP/main/interests.csv
!curl.exe --output interest.csv --url https://raw.githubusercontent.com/tvdermeer/NLP/main/interests.csv
!curl.exe --output stop_words.txt --url https://raw.githubusercontent.com/tvdermeer/NLP/main/stop_words.txt


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 91728  100 91728    0     0  91728      0  0:00:01 --:--:--  0:00:01  220k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  3566  100  3566    0     0   3566      0  0:00:01 --:--:--  0:00:01  9509


Inladen van de interesse-tekstjes. De tekstjes bestaan uit beschreven interesses van jongeren. 

In [36]:
# load and prepare data
data = pd.read_csv('interest.csv', encoding='ISO-8859-1', header=None)
to_lda = data[0]

# load and prepare dutch stop words
stops = pd.read_fwf('stop_words.txt', header=None)
stopwords = stops[0].values.tolist()
data

C:\Users\THOMEER\envs\nlp\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,0
0,2 tussenuren achter elkaar gehad
1,3 minuten gelden ging ik met stijn een app op ...
2,Aan een project gewerkt van bio
3,Aangekleed-haar gedaan
4,Aardrijkskunde en scheikunde geleerd
...,...
1699,Zwemles gegeven aan de kindjes in badje 1.
1700,Zwemles gegeven aan kinderen in badje 1. En ee...
1701,Zwemles gegeven zodat ze diploma kunnen zwemmen.
1702,Zwemles gegeven. De kinderen weer wat nieuws a...


Het maken van een matrix waarin de verschillende soorten woorden per tekstje woorden geteld

In [28]:
# create a co-occurence matrix of the words in the dataset
# in short, words that occur in the same context, tend to have a similar meaning 
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                stop_words = stopwords,
                                max_df = 0.5, 
                                min_df = 10)

tfidf_vectorizer = TfidfVectorizer(**tf_vectorizer.get_params())

dtm_tfidf = tfidf_vectorizer.fit_transform(to_lda)
dtm_tfidf.shape

(1704, 111)

inladen van het LDA model

In [29]:
# creating the LDA model, change the n_components for a different number of topics
lda_tfidf = LatentDirichletAllocation(n_components=20, random_state=0)
lda_tfidf.fit(dtm_tfidf)

LatentDirichletAllocation(n_components=20, random_state=0)

Visualisatie van de topics 

In [30]:
import pyLDAvis
import pyLDAvis.sklearn

# create a visualisation of the topics
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer)

# usage: see the distribution of the topic and hover over the topics to see the words associated in the topic.

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
19     0.262050  0.009581       1        1  6.571233
4      0.292934  0.057449       2        1  5.895504
2     -0.080023 -0.178705       3        1  5.613582
0     -0.053856  0.090580       4        1  5.578023
6      0.236841  0.068430       5        1  5.528778
15    -0.155019  0.273143       6        1  5.415463
9      0.191707 -0.050653       7        1  5.384261
16    -0.128895 -0.133342       8        1  5.302544
14    -0.147943 -0.169727       9        1  5.209464
1     -0.171078  0.191830      10        1  5.046012
12    -0.018002 -0.114337      11        1  4.952695
8      0.090621  0.083180      12        1  4.785860
5     -0.193410  0.088597      13        1  4.754152
11    -0.023401 -0.108845      14        1  4.671382
17    -0.108851 -0.196559      15        1  4.573222
10    -0.116994  0.107401      16        1  4.332281
3     -0.092951  0.076911      17        1  4.165368
18    -0.000905 -0.039779      18        1  4.103593
7     -0.021477 -0.053983      19        1  4.061693
13     0.238651 -0.001172      20        1  4.054892, topic_info=          Term        Freq       Total Category  logprob  loglift
29     gekeken  131.000000  131.000000  Default  30.0000  30.0000
39    gespeeld   50.000000   50.000000  Default  29.0000  29.0000
34     gemaakt   73.000000   73.000000  Default  28.0000  28.0000
24      gedaan   43.000000   43.000000  Default  27.0000  27.0000
28     gegeten   38.000000   38.000000  Default  26.0000  26.0000
..         ...         ...         ...      ...      ...      ...
75        rust    0.056919    9.645949  Topic20  -7.2684  -1.9274
96  verjaardag    0.056919   14.190723  Topic20  -7.2684  -2.3135
27      gegaan    0.056919   34.462505  Topic20  -7.2684  -3.2007
41     gezeten    0.056919   13.482107  Topic20  -7.2684  -2.2622
66      nieuwe    0.056919   26.189975  Topic20  -7.2684  -2.9262

[972 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
0         8  0.879530  aardrijkskunde
1         5  0.922832     afgesproken
2         4  0.861066        allemaal
3        19  0.909749           avond
4        20  0.900324             bed
...     ...       ...             ...
107      16  0.849720          zitten
108       5  0.676932         zojuist
108      14  0.225644         zojuist
109      16  0.909948           zusje
110       4  0.879281         zwemmen

[166 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[20, 5, 3, 1, 7, 16, 10, 17, 15, 2, 13, 9, 6, 12, 18, 11, 4, 19, 8, 14])